In [112]:
import os
import pandas as pd
import unidecode

In [113]:
# Altere o caminho para o caminho na sua máquina
folder_path = r"C:\Users\henrique.figueira\Documents\arquivos_vscode\nba_shooting_analysis\datasets"

df_list = []

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        df_list.append(df)

union_df = pd.concat(df_list, ignore_index=True)

print(union_df)

         SEASON_1 SEASON_2     TEAM_ID               TEAM_NAME  PLAYER_ID  \
0            2004  2003-04  1610612747      Los Angeles Lakers        977   
1            2004  2003-04  1610612757  Portland Trail Blazers        757   
2            2004  2003-04  1610612747      Los Angeles Lakers        977   
3            2004  2003-04  1610612757  Portland Trail Blazers        757   
4            2004  2003-04  1610612757  Portland Trail Blazers        757   
...           ...      ...         ...                     ...        ...   
4231257      2024  2023-24  1610612737           Atlanta Hawks     203992   
4231258      2024  2023-24  1610612737           Atlanta Hawks     203991   
4231259      2024  2023-24  1610612737           Atlanta Hawks     203991   
4231260      2024  2023-24  1610612737           Atlanta Hawks    1627749   
4231261      2024  2023-24  1610612737           Atlanta Hawks    1627749   

               PLAYER_NAME POSITION_GROUP POSITION   GAME_DATE   GAME_ID  .

In [114]:
union_df['PLAYER_NAME'] = (
    union_df['PLAYER_NAME']
    .str.lower()
    .apply(unidecode.unidecode)
    .str.strip()
    .str.replace(r'\s+', ' ', regex=True)
    .str.replace('.', '', regex=False)
    .str.replace("'", '', regex=False)
)

In [115]:
union_df = union_df.drop(columns=['SEASON_1'])
union_df['GAME_DATE'] = pd.to_datetime(union_df['GAME_DATE'], format='%m-%d-%Y', errors='coerce')
union_df.rename(columns={'SEASON_2': 'SEASON'}, inplace=True)

In [116]:
def nba_shots_info():
    temp = pd.DataFrame(index=union_df.columns)
    temp["Datatype"] = union_df.dtypes
    temp["Not null values"] = union_df.count()
    temp["Null Values"] = union_df.isnull().sum()
    temp["Percentage of Null Values"] = (union_df.isnull().mean())*100
    temp["Unique count"] = union_df.nunique()
    temp["Duplicate Values"] = union_df.duplicated().sum()
    return temp
nba_shots_info()

,Datatype,Not null values,Null Values,Percentage of Null Values,Unique count,Duplicate Values
SEASON,object,4231262,0,0.000000,21,192
TEAM_ID,int64,4231262,0,0.000000,30,192
TEAM_NAME,object,4231262,0,0.000000,36,192
PLAYER_ID,int64,4231262,0,0.000000,2162,192
PLAYER_NAME,object,4231262,0,0.000000,2162,192
POSITION_GROUP,object,4223332,7930,0.187415,3,192
POSITION,object,4223332,7930,0.187415,17,192
GAME_DATE,datetime64[ns],4231262,0,0.000000,3354,192
GAME_ID,int64,4231262,0,0.000000,25223,192
HOME_TEAM,object,4231262,0,0.000000,34,192


In [117]:
null_values_mask = union_df['POSITION_GROUP'].isnull()
null_values_rows = union_df[null_values_mask]
null_values_rows_grouped = null_values_rows.groupby('PLAYER_NAME').size()
null_values_rows_grouped

PLAYER_NAME
aj green                321
cam reynolds             96
chance comanche           5
charlie brown jr        111
gg jackson              563
jeff dowtin              26
jeff dowtin jr           93
jeffery taylor          730
jermaine samuels jr      14
john butler jr           53
matt hurt                34
michael frazier ii       29
mj walker                 4
nate williams            50
og anunoby             3417
pj dozier               634
reggie bullock jr        82
tj leaf                 415
trevon scott              6
vitor faverani          147
xavier tillman          585
xavier tillman sr       515
dtype: int64

In [118]:
# Players with over 500 shots and with only one positiong throughout the carrer
union_df.loc[(union_df['PLAYER_NAME'] == 'og anunoby') & (union_df['POSITION_GROUP'].isnull()), 'POSITION_GROUP'] = 'F'
union_df.loc[(union_df['PLAYER_NAME'] == 'og anunoby') & (union_df['POSITION'].isnull()), 'POSITION'] = 'SF'
union_df.loc[(union_df['PLAYER_NAME'] == 'jeffery taylor') & (union_df['POSITION_GROUP'].isnull()), 'POSITION_GROUP'] = 'F'
union_df.loc[(union_df['PLAYER_NAME'] == 'jeffery taylor') & (union_df['POSITION'].isnull()), 'POSITION'] = 'SF'
union_df.loc[(union_df['PLAYER_NAME'] == 'xavier tillman') & (union_df['POSITION_GROUP'].isnull()), 'POSITION_GROUP'] = 'F'
union_df.loc[(union_df['PLAYER_NAME'] == 'xavier tillman') & (union_df['POSITION'].isnull()), 'POSITION'] = 'PF'
union_df.loc[(union_df['PLAYER_NAME'] == 'xavier tillman sr') & (union_df['POSITION_GROUP'].isnull()), 'POSITION_GROUP'] = 'F'
union_df.loc[(union_df['PLAYER_NAME'] == 'xavier tillman sr') & (union_df['POSITION'].isnull()), 'POSITION'] = 'PF'
union_df.loc[(union_df['PLAYER_NAME'] == 'gg jackson') & (union_df['POSITION_GROUP'].isnull()), 'POSITION_GROUP'] = 'F'
union_df.loc[(union_df['PLAYER_NAME'] == 'gg jackson') & (union_df['POSITION'].isnull()), 'POSITION'] = 'PF'

In [119]:
# Players with over 500 shots and with more than one positiong throughout the carrer
union_df.loc[(union_df['PLAYER_NAME'] == 'pj dozier') &
             (union_df['SEASON'] == '2017-18') &
             (union_df['POSITION_GROUP'].isnull()), 'POSITION_GROUP'] = 'G'

union_df.loc[(union_df['PLAYER_NAME'] == 'pj dozier') &
             (union_df['SEASON'] == '2017-18') &
             (union_df['POSITION'].isnull()), 'POSITION'] = 'PG'

seasons_list = ['2018-19', '2019-20', '2020-21', '2021-22']

union_df.loc[(union_df['PLAYER_NAME'] == 'pj dozier') & 
              (union_df['SEASON'].isin(seasons_list)) & 
              (union_df['POSITION_GROUP'].isnull()), 'POSITION_GROUP'] = 'G'

union_df.loc[(union_df['PLAYER_NAME'] == 'pj dozier') & 
              (union_df['SEASON'].isin(seasons_list)) & 
              (union_df['POSITION'].isnull()), 'POSITION'] = 'sG'

In [120]:
# Team names changed
union_df['TEAM_NAME'] = union_df['TEAM_NAME'].replace('LA Clippers', 'Los Angeles Clippers')
union_df['TEAM_NAME'] = union_df['TEAM_NAME'].replace('New Orleans/Oklahoma City Hornets', 'New Orleans Hornets')

In [121]:
# Cleaning process completed
def nba_shots_info():
    temp = pd.DataFrame(index=union_df.columns)
    temp["Datatype"] = union_df.dtypes
    temp["Not null values"] = union_df.count()
    temp["Null Values"] = union_df.isnull().sum()
    temp["Percentage of Null Values"] = (union_df.isnull().mean())*100
    temp["Unique count"] = union_df.nunique()
    temp["Duplicate Values"] = union_df.duplicated().sum()
    return temp
nba_shots_info()

,Datatype,Not null values,Null Values,Percentage of Null Values,Unique count,Duplicate Values
SEASON,object,4231262,0,0.00000,21,192
TEAM_ID,int64,4231262,0,0.00000,30,192
TEAM_NAME,object,4231262,0,0.00000,34,192
PLAYER_ID,int64,4231262,0,0.00000,2162,192
PLAYER_NAME,object,4231262,0,0.00000,2162,192
POSITION_GROUP,object,4229776,1486,0.03512,3,192
POSITION,object,4229776,1486,0.03512,18,192
GAME_DATE,datetime64[ns],4231262,0,0.00000,3354,192
GAME_ID,int64,4231262,0,0.00000,25223,192
HOME_TEAM,object,4231262,0,0.00000,34,192


In [122]:
# union_df.to_csv('nba.csv', index=False)